# Where is the best place to settle in as a HEC Montreal new student ?

## Summary

1. Introduction / Business problem  
2. Getting and cleaning data  
3. First visualization and analysis  
4. Clustering and refining data  
   1. DBSCAN clustering  
   2. Getting complementary data  
5. Results      
6. Discussion  
7. Conclusion  

## 1. Introduction / Business problem

This is a true-life problem I would like to solve. 

My daughter started studying at HEC Montreal (Quebec, Canada) but due to the Covid-19 pandemic, she could not move there from France so far. She may be able to go there next summer and will then have to find a flat or room. She would like to find the best zone, close to her school, where she could find:

- french and italian retaurants (she's French and loves italian food ;-) )
- a park
- a pool
- a supermarket
- a metro station

The aim is to show these places on a map, with different color for each category, and determine the zones that fulfill most criteria. 
Since parks are already displayed on the map, I chose not to search them in Foursquare.

## 2. Getting and cleaning data

The datasets will come from the following Foursquare categories as required for this work. The corresponding Foursquare category ids are between brackets:

- French Restaurant  (4bf58dd8d48988d10c941735)
- Italian Restaurant  (4bf58dd8d48988d110941735)
- Pool  (4bf58dd8d48988d15e941735)
- Metro Station  (4bf58dd8d48988d1fd931735)
- Supermarket   (52f2ab2ebcbc57f1066b8b46

The area is centered on HEC Montreal coordinates, with a radius of 4 000 m. 

Let's get and clean the data:

In [1]:
#coding = utf-8
import numpy as np
import pandas as pd
import requests
import warnings; warnings.simplefilter('ignore') # to avoid warnings on slicing

In [3]:
# client_id = ....
# client_secret = ...
endpoint = "https://api.foursquare.com/v2/venues/search"
v = "20200101"
hec_ll = "45.50433081907987,-73.62086000795014"         # HEC MONTREAL coordinates
radius = str(4000)   # radius of search around central point, in meters
limit = str(100)     # max number of venues returned by the request
query = f'{endpoint}?client_id={client_id}&client_secret={client_secret}&v={v}&ll={hec_ll}&radius={radius}&limit={limit}&categoryId='

In [4]:
categories = {
    'french': "4bf58dd8d48988d10c941735",
    'italian':  "4bf58dd8d48988d110941735",
    'pools':  "4bf58dd8d48988d15e941735",
    'metros':  "4bf58dd8d48988d1fd931735",
    'supermarkets':  "52f2ab2ebcbc57f1066b8b46"}

list_of_dicts = []
for cat_name, cat_id in categories.items():
    query_for_category = query + cat_id
    list_of_venues_for_category = requests.get(query_for_category).json()['response']['venues']
    for venue in list_of_venues_for_category:
        list_of_dicts.append({
            'id': venue['id'],
            'name': venue['name'],
            'lat': venue['location']['lat'],
            'lon': venue['location']['lng'],
            'address': venue['location']['formattedAddress'][0],
            'category': cat_name
        })
venues = pd.DataFrame(list_of_dicts)

Here is our base dataframe:

In [5]:
venues

,id,name,lat,lon,address,category
0,4b6c85fdf964a520ad402ce3,Duc de Lorraine,45.494618,-73.618279,5002 ch. de la Côte-des-Neiges (Rue du Frère A...,french
1,4adf33eef964a520497821e3,Kitchen Galerie,45.535375,-73.617076,60 Jean-Talon E. (St Dominique),french
2,5c3cd1a4f2554e002c2d4bd9,Restaurant Jérôme Ferrer – Europea,45.496492,-73.571741,"1065, rue de la Montagne",french
3,4de03e6745dd3eae8769f11b,L'Acadamie,45.517964,-73.576169,Brossard QC,french
4,5cb7358c01bc5a002cfce800,Shaker St-Laurent,45.513056,-73.571077,3518 Boul Saint-Laurent (Milton),french
...,...,...,...,...,...,...
205,4ae1f65ef964a5202f8921e3,Metro Avenue du Parc,45.510307,-73.575101,3575 Av Du Parc Suite 5100 (Les Galeries du Parc),supermarkets
206,4b3bd71df964a520527c25e3,Provigo,45.476032,-73.622829,5595 avenue Monkland,supermarkets
207,4e40861ba8099680852c6769,Provigo,45.501104,-73.628182,Canada,supermarkets
208,4bb63eec6edc76b08aa0301c,Metro Mont-Royal,45.520474,-73.624400,1280 Av Beaumont,supermarkets


Check the number of each category (limited to 100 in the request)

In [6]:
venues.groupby("category").count()[['id']]

,id
category,
french,49
italian,50
metros,31
pools,45
supermarkets,35


Let's create a different dataframe for each category and add colors in view of future map creation. The dataframes will be named *df_french*, *df_italian*, etc.

In [7]:
list_of_dfs = []
colors = ['red', 'green', 'blue', 'black', 'orange']
for category, color in zip(categories.keys(), colors):
    df_name = "df_" + category
    globals()[df_name] = venues[venues['category']==category]
    globals()[df_name].loc[:,'color'] = color
    list_of_dfs.append(globals()[df_name])

In [8]:
df_french.head()

,id,name,lat,lon,address,category,color
0,4b6c85fdf964a520ad402ce3,Duc de Lorraine,45.494618,-73.618279,5002 ch. de la Côte-des-Neiges (Rue du Frère A...,french,red
1,4adf33eef964a520497821e3,Kitchen Galerie,45.535375,-73.617076,60 Jean-Talon E. (St Dominique),french,red
2,5c3cd1a4f2554e002c2d4bd9,Restaurant Jérôme Ferrer – Europea,45.496492,-73.571741,"1065, rue de la Montagne",french,red
3,4de03e6745dd3eae8769f11b,L'Acadamie,45.517964,-73.576169,Brossard QC,french,red
4,5cb7358c01bc5a002cfce800,Shaker St-Laurent,45.513056,-73.571077,3518 Boul Saint-Laurent (Milton),french,red


## 3. First visualization and analysis

We will use Folium librairy to create a map centered on HEC Montréal.

In [9]:
import folium
hec_lat_lon = [float(item) for item in hec_ll.split(',')]

We define a class that will help us to display markers:

In [10]:
class VenuesMap:
    
    def __init__(self):
        self.venues_map = folium.Map(location = hec_lat_lon, zoom_start=13)
        folium.Marker(hec_lat_lon, popup='<b>HEC Montreal</b>', tooltip = "HEC Montreal").add_to(self.venues_map)

    def add_markers_for(self, dfs, cluster=False, opacity=1, edge_color='black'):
        for df in dfs:
            if cluster:
                zipped_list = zip(df['lat'], df['lon'], df['cluster color'], df['name'], df['category'])
                radius = 10
            else:
                zipped_list = zip(df['lat'], df['lon'], df['color'], df['name'], df['category'])
                radius = 5
            for lat, lon, color, name, category in zipped_list:
                folium.CircleMarker(
                    [lat, lon],
                    radius=radius,
                    weight=1,
                    color=edge_color,
                    popup=f'{name} ({category})',
                    fill=True,
                    fill_color=color,
                    fill_opacity=opacity).add_to(self.venues_map)
    
    def show(self):
        display(self.venues_map)
        
    reset = __init__

venues_map = VenuesMap()

First, let's have a look at the map without any venue, only HEC Montreal marker:

In [11]:
venues_map.show()

Now let's add markers for all venues we are interested in:
- french restaurants in red
- italian restaurants in green
- pools in blue
- metro in black
- supermarket in orange

In [12]:
venues_map.add_markers_for(list_of_dfs)
venues_map.show()

A first conclusion is this map is not very clear. We would like to synthetize data by clustering some categories in several clusters so the final map is easier to understand.  

This is especially important for venues such as restaurants : usually they tend to aggregate in certain places, so a clustering is likely to show the more animated neighborhoods of the city. While clustering pools, supermarkets or metro stations is not really relevant from that point of view : they are more utilitarian facilities that have to be equally spread on the territory.

## 4. Clustering and refining data

Let's start by visualizing only french restaurants:

In [13]:
venues_map.reset()
venues_map.add_markers_for([df_french])
venues_map.show()

Are the italian restaurants grouped the same way ?

In [14]:
venues_map.reset()
venues_map.add_markers_for([df_italian])
venues_map.show()

It seems that they are distributed differently. So let's cluster them separately.

### A. DBSCAN clustering

We could use k-means clustering which is very commonly used, and efficient. But there are outliers that may alter this k-means clustering, such as the restaurants located close to the Autoroute Décarie for instance, 

Furthermore, the notion of density is important : the more restaurants are in a given area, the more it can be considered as an lively and nice place. This is why DBSCAN, based on denity analysis, may better fit our purpose.

In [15]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

Let's find clusters for french restaurants:

In [16]:
X_french = df_french[['lat', 'lon']].to_numpy()
X_french = StandardScaler().fit_transform(X_french)
clustering = DBSCAN().fit(X_french)
print('The list of labels is:')
print(clustering.labels_)
print(f'There are {len(set(clustering.labels_))} clustering labels:')
print(set(clustering.labels_))

The list of labels is:
[-1 -1  0  0  0  0  0 -1  0 -1  0 -1  0  0  0  0  0  0  0  0  0  0  0 -1
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0 -1  0  0 -1
  0]
There are 2 clustering labels:
{0, -1}


Add color label according to clustering label, we chose red and pink since there are 2 labels:

In [17]:
cluster_color = [['red', 'pink'][color+1] for color in clustering.labels_]
df_french['cluster color'] = cluster_color
df_french.head()

,id,name,lat,lon,address,category,color,cluster color
0,4b6c85fdf964a520ad402ce3,Duc de Lorraine,45.494618,-73.618279,5002 ch. de la Côte-des-Neiges (Rue du Frère A...,french,red,red
1,4adf33eef964a520497821e3,Kitchen Galerie,45.535375,-73.617076,60 Jean-Talon E. (St Dominique),french,red,red
2,5c3cd1a4f2554e002c2d4bd9,Restaurant Jérôme Ferrer – Europea,45.496492,-73.571741,"1065, rue de la Montagne",french,red,pink
3,4de03e6745dd3eae8769f11b,L'Acadamie,45.517964,-73.576169,Brossard QC,french,red,pink
4,5cb7358c01bc5a002cfce800,Shaker St-Laurent,45.513056,-73.571077,3518 Boul Saint-Laurent (Milton),french,red,pink


Then let's visualize the map with clusters:

In [18]:
venues_map.reset()
venues_map.add_markers_for([df_french], cluster=True)
venues_map.show()

The density criteria is respected : pink retaurants are in the most dense place, clumped together, while red ones are part of a sparser cluster.

Now let's apply the same process to italian restaurants:

In [19]:
X_italian = df_italian[['lat', 'lon']].to_numpy()
X_italian = StandardScaler().fit_transform(X_italian)
clustering = DBSCAN().fit(X_italian)
print('The list of labels is:')
print(clustering.labels_)
print(f'There are {len(set(clustering.labels_))} clustering labels:')
print(set(clustering.labels_))

The list of labels is:
[ 0  1  0  1  2  1  2  2  2 -1  1  1  0  2  2  2  0  2 -1  0  0  0  2  0
  1  0  2  2  0  2  2 -1 -1  2  2  0  2  2  0  2  1  1  0 -1  2 -1 -1  2
 -1  2]
There are 4 clustering labels:
{0, 1, 2, -1}


In [20]:
cluster_color = [['blue', 'green', 'lightblue', 'lightgreen'][color+1] for color in clustering.labels_]
df_italian['cluster color'] = cluster_color
df_italian.head()

,id,name,lat,lon,address,category,color,cluster color
49,4adb932bf964a5201b2921e3,Caffè Italia,45.532862,-73.614178,6840 boul. Saint-Laurent,italian,green,green
50,5942ea0dc21cb154610b4aa8,Il Miglio,45.524270,-73.595449,5235 Saint-Laurent,italian,green,lightblue
51,4be5f341bcef2d7f4e9604e5,Cafe Via Dante,45.534934,-73.611993,251 rue Dante,italian,green,green
52,4ddfd638c65bb5e319dbad0a,Il Pagliaccio,45.519284,-73.596082,365 Laurier O (Parc),italian,green,lightblue
53,4bc1186b4cdfc9b686279421,Chez Ennio,45.492675,-73.582546,1978 boul. de Maisonneuve Ouest (Fort),italian,green,lightgreen


In [21]:
venues_map.reset()
venues_map.add_markers_for([df_italian], cluster=True)
venues_map.show()

Again, DBSCAN shows its efficiency : green clusters are those of aggregated retaurants, while blue cluster is that of sparse ones.

Now we can visualize both kind of retaurants on the same map :

In [22]:
venues_map.reset()
venues_map.add_markers_for([df_french, df_italian], cluster=True)
venues_map.show()

We see that 3 zones concentrate both french and italian restaurants:
- the light blue and southern part of pink circles (close to Villemarie)
- the light green, dark blue, and northern part of pink circles (close to Mont-Royal)
- the grey blue and red circles (close to Villeray Saint Michel Parc Extension)

Let's unify their colors and make them translucent, then add metro stations (in black), pools (blue) and supermarkets (orange):

In [23]:
df_french['cluster color'] = "red"
df_italian['cluster color'] = "green"
df_restaurants = pd.concat([df_french, df_italian])


In [24]:
venues_map.reset()
venues_map.add_markers_for([df_restaurants], cluster=True, opacity=0.2, edge_color=None)
venues_map.add_markers_for([df_pools, df_metros, df_supermarkets])
venues_map.show()

### B. Getting complementary data

There are lots of metro station, but only those on the Blue line go to HEC Montreal. So we'd like to narrow the list of stations to these ones.

Is color of line supplied by Foursquare ? Let's query again for metros and check the response part of json:

In [25]:
query_for_metros = query + categories['metros']
metros = requests.get(query_for_metros)
metros.json()['response']['venues'][0]

{'id': '4b2abacdf964a52078ae24e3',
 'name': 'STM Station Université-de-Montréal',
 'location': {'lat': 45.50346642237186,
  'lng': -73.61810340775273,
  'labeledLatLngs': [{'label': 'display',
    'lat': 45.50346642237186,
    'lng': -73.61810340775273}],
  'distance': 235,
  'cc': 'CA',
  'city': 'Montréal',
  'state': 'QC',
  'country': 'Canada',
  'formattedAddress': ['Montréal QC', 'Canada']},
 'categories': [{'id': '4bf58dd8d48988d1fd931735',
   'name': 'Metro Station',
   'pluralName': 'Metro Stations',
   'shortName': 'Metro',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/subway_',
    'suffix': '.png'},
   'primary': True}],
 'referralId': 'v-1607166274',
 'hasPerk': False}

The metro line color is not indicated, so we have to find the list of stations for the Blue line elsewhere. Let's try with Wikipedia:

In [26]:
wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_Montreal_Metro_stations')
wiki[4]

,0,1
0,Green,Angrignon Monk Jolicoeur Verdun De L'Église La...
1,Orange,Côte-Vertu Du Collège De La Savane Namur Plamo...
2,Yellow,Berri–UQAM Jean-Drapeau Longueuil–Université-d...
3,Blue,Snowdon Côte-des-Neiges Université-de-Montréal...


The last line is what we need, let's split it (and fix the splitting of "Parc de Castelnau")

In [27]:
blue_stations = wiki[4].iloc[3,1].split()
blue_stations[6] = "Parc de Castelnau"
cleaned = [item for item in blue_stations if item not in ["Parc","De", "Castelnau"]]
blue_stations = cleaned
blue_stations

['Snowdon',
 'Côte-des-Neiges',
 'Université-de-Montréal',
 'Édouard-Montpetit',
 'Outremont',
 'Acadie',
 'Parc de Castelnau',
 'Jean-Talon',
 'Fabre',
 "D'Iberville",
 'Saint-Michel']

Now let's find, in df_metros, which ones are on Blue line:

In [28]:
is_blue = []
for station in df_metros['name']:
    is_blue.append(any([blue_station in station for blue_station in blue_stations]))
df_metros['blue line'] = is_blue
df_metros.head()

,id,name,lat,lon,address,category,color,blue line
144,4b2abacdf964a52078ae24e3,STM Station Université-de-Montréal,45.503466,-73.618103,Montréal QC,metros,black,True
145,4b6caf0af964a520f94b2ce3,STM Station de L'Acadie,45.525726,-73.625000,Montréal QC,metros,black,True
146,4aff627cf964a520063822e3,STM Station de Snowdon,45.485333,-73.627860,Montréal QC,metros,black,True
147,4b57256ff964a520372828e3,STM Station de la Côte-des-Neiges,45.496377,-73.622564,Montréal QC,metros,black,True
148,4b707791f964a520391c2de3,STM Station Plamondon,45.494280,-73.637959,Station Plamondon,metros,black,False


And keep only them:

In [29]:
index_not_on_blue_line = df_metros[df_metros['blue line'] == False].index
df_metros.drop(index_not_on_blue_line, inplace=True)
df_metros

,id,name,lat,lon,address,category,color,blue line
144,4b2abacdf964a52078ae24e3,STM Station Université-de-Montréal,45.503466,-73.618103,Montréal QC,metros,black,True
145,4b6caf0af964a520f94b2ce3,STM Station de L'Acadie,45.525726,-73.625000,Montréal QC,metros,black,True
146,4aff627cf964a520063822e3,STM Station de Snowdon,45.485333,-73.627860,Montréal QC,metros,black,True
147,4b57256ff964a520372828e3,STM Station de la Côte-des-Neiges,45.496377,-73.622564,Montréal QC,metros,black,True
150,4b67509ef964a52070472be3,STM Station Édouard-Montpetit,45.509822,-73.612816,Station Édouard-Montpetit,metros,black,True
172,4b0403c4f964a520e55022e3,STM Station Jean-Talon,45.538999,-73.613767,Station Jean-Talon,metros,black,True
173,4b9a9fe5f964a5206ac735e3,STM Station d'Outremont,45.517464,-73.612114,1400 Van horne,metros,black,True


## 5. Results

Let's draw the final map with restaurants, pools, supermarkets, and only blue line metro stations. To improve visibility, the metro station markers are colored in red:

In [30]:
df_metros['color'] = 'red'
venues_map.reset()
venues_map.add_markers_for([df_restaurants], cluster=True, opacity=0.2, edge_color=None)
venues_map.add_markers_for([df_pools, df_metros, df_supermarkets])
venues_map.show()

There is no Blue line station is on the Ville-Marie zone.  
The Outremont station is a little far from the restaurants of Le Plateau - Mont-Royal.   
But the North zone, ***Villeray-Saint-Michel-Parc-extension*** fulfills all required criteria:
- french and italian restaurants
- Blue line metro station
- close to a park (parc Jarry, as can be seen on the map)
- close to a supermarket (there is one just next the metro station)
- close to a swimming pool (Piscine St Denis which is just next the metro station, or Piscine Jarry).



## 6. Discussion

The clustering did not take in account parameters such as, for restaurants, the prices. This could have been an interesting approach too, but the aim was essentially to spot geographic clusters. This work is based on the assumption that what makes a neighborhood lively is the high number of leisure venues such as restaurants. 

Bars and shops could also have been added, but for bars, my daughter will discover them soon enough by herself ;-), and for shops, they may not be that representative of lively places where students like to come together.  

Another important element that was not taken in account was rental prices, and we may discover that prices are very high (or suspiciously low) in the selected zone. This analysis may be carried out to continue the present one.

# 7. Conclusion

The analysis of Foursquare data in conjunction with map rendering allowed to answer the question "What is the best place to settle to be close to french and italian restaurants, a pool, a park, a metro station linked to HEC Montréal, and a supermarket?". And the answer is likely to be **Villeray-Saint-Michel**. 

But "a map is not the territory it represents" (Alfred Korybski) and many other parameters may invalidate this conclusion. So a battle-hardened data scientist may smile while reading this little report, but as a final exercice, I did enjoy building it and to be able to define my own problem and find my own solutions.

# 8. References

[Foursquare API](https://developer.foursquare.com/)  
[Wikipedia page "List of Montreal Metro stations"](https://en.wikipedia.org/wiki/List_of_Montreal_Metro_stations)